In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df  =  pd.read_csv('covid_toy.csv')

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        100 non-null    int64  
 1   gender     100 non-null    object 
 2   fever      90 non-null     float64
 3   cough      100 non-null    object 
 4   city       100 non-null    object 
 5   has_covid  100 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ KB


In [6]:
df.columns

Index(['age', 'gender', 'fever', 'cough', 'city', 'has_covid'], dtype='object')

In [7]:
df['gender'].value_counts()

Female    59
Male      41
Name: gender, dtype: int64

In [8]:
df['cough'].value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [9]:
df['city'].value_counts()

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

In [10]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test  =  train_test_split(df.drop(columns=['has_covid']), df['has_covid'], test_size=0.2, random_state =1)

In [12]:
X_train

,age,gender,fever,cough,city
33,26,Female,98.0,Mild,Kolkata
18,64,Female,98.0,Mild,Bangalore
88,5,Female,100.0,Mild,Kolkata
38,49,Female,101.0,Mild,Delhi
30,15,Male,101.0,Mild,Delhi
...,...,...,...,...,...
61,81,Female,98.0,Strong,Mumbai
58,23,Male,98.0,Strong,Mumbai
49,44,Male,104.0,Mild,Mumbai
26,19,Female,100.0,Mild,Kolkata


In [13]:
X_train.shape

(80, 5)

In [14]:
y_train.shape

(80,)

In [15]:
SI  =  SimpleImputer()
X_train_fever = SI.fit_transform(X_train[['fever']])

X_test_fever = SI.fit_transform(X_test[['fever']])

# simple imputer

In [18]:
X_test_fever.shape

(20, 1)

In [19]:
# encoding
# ordinalencoding
OE  =  OrdinalEncoder(categories = [['Mild', 'Strong']])

X_trainCough  = OE.fit_transform(X_train[['cough']])


X_testCough  = OE.fit_transform(X_test[['cough']])

In [21]:
X_trainCough.shape

(80, 1)

In [22]:
X_testCough.shape

(20, 1)

In [23]:
# OneHot encoding
# total 6 col, 2 + 4
OHE  =  OneHotEncoder(drop='first', sparse= False)

X_trainGenderCity  =  OHE.fit_transform(X_train[['gender','city']])


X_testGenderCity  =  OHE.fit_transform(X_test[['gender','city']])

In [24]:
X_trainGenderCity.shape

(80, 4)

In [25]:
X_trainAge  =  X_train.drop(columns=['gender','fever','cough','city']).values

X_testAge  =  X_test.drop(columns=['gender','fever','cough','city']).values

In [27]:
X_trainAge.shape

(80, 1)

In [28]:
X_train_transform  = np.concatenate((X_trainAge,X_trainGenderCity, X_trainCough, X_train_fever ), axis=1)

In [29]:
X_test_transform  = np.concatenate((X_testAge,X_testGenderCity, X_testCough, X_test_fever ), axis=1)

In [31]:
X_train_transform.shape

(80, 7)

In [32]:
# using column transformer

from sklearn.compose import ColumnTransformer

In [35]:
transformer  =  ColumnTransformer(transformers=[
    ('trans1', SimpleImputer(),['fever']),
    ('trans2', OrdinalEncoder(categories=[['Mild','Strong']]), ['cough']),
    ('trans3',OneHotEncoder(sparse=False, drop='first'),['gender','city'])
], remainder='passthrough')

In [36]:
transformer.fit_transform(X_train)

array([[ 98.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  26.        ],
       [ 98.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  64.        ],
       [100.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   5.        ],
       [101.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  49.        ],
       [101.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  15.        ],
       [100.91666667,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  82.        ],
       [ 98.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  80.        ],
       [100.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  19.        ],
       [ 99.        ,   1.        ,   0.        ,   1.        ,
          0.    

In [37]:
transformer.fit_transform(X_train).shape

(80, 7)

In [39]:
transformer.transform(X_test).shape

(20, 7)